In [0]:
! pip install mxnet-cu92

In [0]:
import mxnet as mx
from mxnet import autograd, nd, init, gluon
from mxnet.gluon import nn, loss as gloss, data as gdata

import os
import shutil
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import zipfile

In [10]:
from google.colab import drive
drive.mount('/content/gdrive')

data_path = '/content/gdrive/My\ Drive/Datasets/dogcat'
base_path = '/content/dogcat'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [47]:
! cp -r $data_path /content
! ls /content/dogcat

sample_submission.csv  testset.zip  trainset.zip


In [0]:
for f in ['trainset.zip', 'testset.zip']:
    with zipfile.ZipFile(os.path.join(base_path, f)) as z:
        z.extractall(base_path)

In [61]:
base_path = '/content/dogcat'
train_valid_path = 'trainset'
test_path = 'testset'
train_path = 'train'
valid_path = 'valid'


def delete_dir(path):
    if os.path.exists(os.path.join(*path)):
        shutil.rmtree(os.path.join(*path))
        
def create_dir(path):
    if not os.path.exists(os.path.join(*path)):
        os.makedirs(os.path.join(*path))

def devide_train_valid(base_path, train_valid_path, train_path, valid_path,
                       valid_ratio=0.2, labels=['Dog', 'Cat']):
    delete_dir([base_path, train_path])
    delete_dir([base_path, valid_path])
        
    all_files_by_label = [os.listdir(os.path.join(base_path, train_valid_path, label))
                          for label in labels]
    
    count_train = {label:0 for label in labels}
    count_valid = {label:0 for label in labels}
    
    for label, file_per_label in zip(labels, all_files_by_label):
        print('Train&Valid set, %s: %s' % (label, len(file_per_label)))
        create_dir([base_path, train_path, label])
        create_dir([base_path, valid_path, label])
        for f in file_per_label:
            if count_train[label] < len(file_per_label) * (1 - valid_ratio):
                shutil.copy(os.path.join(base_path, train_valid_path, label, f),
                            os.path.join(base_path, train_path, label, f))
                count_train[label] += 1
            else:
                shutil.copy(os.path.join(base_path, train_valid_path, label, f),
                            os.path.join(base_path, valid_path, label, f))
                count_valid[label] += 1
        print('Train set, %s: %s' % (label, count_train[label]))
        print('Valid set, %s: %s' % (label, count_valid[label]))
                
devide_train_valid(base_path, train_valid_path, train_path, valid_path)


Train&Valid set, Dog: 9999
Train set, Dog: 8000
Valid set, Dog: 1999
Train&Valid set, Cat: 9999
Train set, Cat: 8000
Valid set, Cat: 1999


In [0]:
batch_size = 256

In [68]:
aug_train = gdata.vision.transforms.Compose([
#     gdata.vision.transforms.RandomResizedCrop(, scale=(0.75, 1),
#                                               ratio=(3.0/4.0, 4.0/3.0)),
    gdata.vision.transforms.RandomFlipLeftRight(),
    gdata.vision.transforms.RandomColorJitter(brightness=0.4, 
                                              contrast=0.4, saturation=0.4),
    gdata.vision.transforms.RandomLighting(0.1),
    gdata.vision.transforms.ToTensor(),
    gdata.vision.transforms.Normalize([0.485, 0.456, 0.406], 
                                      [0.229, 0.224, 0.225])
    
])

aug_test = gdata.vision.transforms.Compose([
    gdata.vision.transforms.ToTensor(),
    gdata.vision.transforms.Normalize([0.485, 0.456, 0.406], 
                                      [0.229, 0.224, 0.225])
])

train_data = gdata.vision.ImageFolderDataset(
    os.path.join(base_path, train_path), flag=1)
valid_data = gdata.vision.ImageFolderDataset(
    os.path.join(base_path, valid_path), flag=1)
train_valid_data = gdata.vision.ImageFolderDataset(
    os.path.join(base_path, train_valid_path), flag=1)
test_data = gdata.vision.ImageFolderDataset(
    os.path.join(base_path, test_path), flag=1)

train_iter = gdata.DataLoader(train_data.transform_first(aug_train), batch_size,
                              shuffle=True, last_batch='keep')
valid_iter = gdata.DataLoader(valid_data.transform_first(aug_test), batch_size,
                              shuffle=True, last_batch='keep')
train_valid_iter = gdata.DataLoader(train_valid_data.transform_first(aug_train), batch_size,
                              shuffle=True, last_batch='keep')
test_iter  = gdata.DataLoader(test_data.transform_first(aug_test), batch_size,
                              shuffle=False, last_batch='keep')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Ignoring /content/dogcat/trainset/.DS_Store, which is not a directory.


In [0]:
for X, y in valid_iter:
    aa, bb = X, y
    break

In [80]:
bb


[0 1 0 0 1 1 1 0 0 0 0 1 1 1 0 1 1 0 1 1 0 0 0 0 1 1 1 1 0 1 1 1 1 0 1 1 1
 0 1 0 0 1 0 0 0 1 1 0 1 1 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 0 1 0 1 1 0 1
 0 0 0 0 0 1 1 0 0 1 1 0 1 1 0 1 0 0 0 0 0 1 1 1 1 1 1 0 0 0 1 0 1 1 0 1 0
 1 0 1 0 0 1 0 0 0 1 1 0 0 0 0 1 1 0 0 1 1 1 1 1 0 1 0 0 1 1 1 0 0 1 0 0 1
 0 0 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 1 0 0 0
 1 1 1 1 1 1 1 1 0 0 0 1 0 1 1 0 0 1 1 1 0 0 0 1 0 1 1 1 1 1 0 1 0 1 1 1 1
 0 0 0 1 1 0 1 0 0 0 1 1 1 0 1 1 0 0 0 0 1 1 0 1 1 0 1 1 1 0 1 1 0 1]
<NDArray 256 @cpu(0)>

## Define ResNet

In [0]:
class ResBlock(nn.HybridBlock):
    def __init__(self, n_channels, strides=1, nin=False):
        super(ResBlock, self).__init__()
        
        self.conv1 = nn.Conv2D(n_channels, kernel_size=3, padding=1, strides=strides)
        self.conv2 = nn.Conv2D(n_channels, kernel_size=3, padding=2, strides=strides)
        
        self.nin   = None
        if nin:
            self.nin = nn.Conv2D(n_channels, kernel_size=1, strides=strides)
            
    def hybride_forward(self, F, X):
        res = F.relu(self.conv1(X))
        res = self.conv2(res)
        
        if self.nin:
            X = self.nin(X)
            
        return F.relu(res + X)

In [0]:
def multiple_block(n_channels, n_blocks, nin=False):
    net = nn.HybridSequential()
    
    if nin:
        net.add(ResBlock(n_channels, 2, True))
        n_blcoks -= 1
    
    for i in range(n_blocks):
        net.add(ResBlock(n_channels))
        
    return net

class ResNet18(nn.HybridBlock):
    
    def __init